# Simulación de variables aleatorias discretas
    

In [ ]:
from numpy import exp, log
import numpy as np
from random import random
import matplotlib.pyplot as plt
from timeit import timeit as timeit


<font color = green size = 5pt>
Ordenamiento de las probabilidades de mayor a menor
</font>

In [ ]:
def TInversa_sin_ordenar(u):
    if u<0.20:
        return 1
    elif u<0.35:
        return 2
    elif u<0.60:
        return 3
    else:
        return 4

In [ ]:
def TInversa_ordenados(u):
    if u<0.40:
        return 4
    elif u<0.65:
        return 3
    elif u<0.80:
        return 1
    else:
        return 2

In [ ]:
### Comparación de tiempos:
n = 100
no_ordenados = [0] * n
ordenados = [0] * n


for u in range(n):
    no_ordenados[u] = timeit("TInversa_sin_ordenar(u/n)", number = 100000, globals = globals())
    ordenados[u] = timeit(setup = "TInversa_ordenados(u/n)", number = 100000, globals = globals())

In [ ]:
fig, ax = plt.subplots(figsize = (15,3))
ax.plot(no_ordenados, 'r', label = 'sin ordenar')
ax.plot(ordenados, 'b', label = 'ordenados')
ax.set_xticklabels([str(i/5.) for i in range(6)])
ax.set_xticks([i for i in range(0,101,20)])
ax.set_xlabel('u')
ax.set_ylabel('tiempo por ejecución')

ax.legend(loc = 'best')

plt.suptitle('Comparación de tiempos de corrida')

<font color = green size = 6pt>
   Binomial
    </font>

In [ ]:
def Binomial1(n,p):
    'Método de transformada inversa'
    U = random()
    i = 0
    c= p / (1-p)
    prob = (1-p) ** n
    F = prob
    while U >= F:
        prob = c * (n-i) / (i+1) * prob
        F= F + prob 
        i = i + 1
    return i


def Binomial2(n, p):
    'Método de TI seleccionando p < 0.5'
    if p > 0.5:
        return n - Binomial1(n, 1-p)
    else:
        return Binomial1(n, p)
    

    

In [ ]:
'''Medición de tiempos de ejecución'''
n = 100
bin1 = [0] * n
bin2 = [0] * n
N = 100000
for p in range(1,n):
    bin1[p] = timeit("Binomial1(15, p/n)", globals = globals(), number = N)
    bin2[p] = timeit("Binomial2(15, p/n)", globals = globals(), number = N)



In [ ]:
fig, ax = plt.subplots(figsize = (15,4))
ax.plot(bin1, 'r', label = 'Binomial normal')
ax.plot(bin2, 'b', label = 'Binomial selectiva')
ax.set_xlabel('p')
ax.set_ylabel('tiempo por ejecución')
ax.set_xticklabels([str(i/5.) for i in range(6)])
ax.set_xticks([i for i in range(0,101,20)])

ax.legend(loc = 'best')
ax.set_title('Binomial(15, p)')
#plt.suptitle('Comparación de tiempos de corrida')

In [ ]:
from scipy.stats import binom
def tasas_riesgo_binomial(n, p):
    t_riesgo = np.empty((n+1))
    probs = np.asarray([binom(n,p).pmf(i) for i in range(n+1)])
    
    condicional = 1.
    for i in range(n+1):
        t_riesgo[i] = probs[i] / condicional
        condicional -= probs[i]
        
    return t_riesgo


def Binomial_TR(n, p, t_riesgo):
    x = 0
    while True:
        U = random()
        if U < t_riesgo[x]:
            return x
        else:
            x += 1
            



In [ ]:
'''Medición de tiempos de ejecución'''
n = 100
bin1 = [0] * n
bin2 = [0] * n
bin3 = [0] * n
N = 10000
for p in range(1,n):
    bin1[p] = timeit("Binomial1(15, p/n)", globals = globals(), number = N)
    bin2[p] = timeit("Binomial2(15, p/n)", globals = globals(), number = N)
    t_riesgo = tasas_riesgo_binomial(15,p/n)
    bin3[p] = timeit("Binomial_TR(15, p/n, t_riesgo)", globals = globals(), number = N)

In [ ]:
fig, ax = plt.subplots(figsize = (15,4))
ax.plot(bin1, 'r', label = 'Binomial normal')
ax.plot(bin2, 'b', label = 'Binomial selectiva')
ax.plot(bin3, 'g', label = 'Binomial Tasas de riesgo')
ax.set_xlabel('p')
ax.set_ylabel('tiempo por ejecución')
ax.set_xticklabels([str(i/5.) for i in range(6)])
ax.set_xticks([i for i in range(0,101,20)])

ax.legend(loc = 'best')
ax.set_title('Binomial(15, p)')
#plt.suptitle('Comparación de tiempos de corrida')

   <font color = green size = 6pt>
   Geométrica
    </font>

In [ ]:
def geom1(p):  
    '''aplicando Transformada inversa'''
    q = 1-p
    F = p
    i = 1
    U = random()
    while U >= F:
        p *= q
        F += p
        i += 1
    return i

def geom2(p):  
    ''' utilizando log(1-U) / log(1-p)'''
    U = random()
    return int(log(1-U) / log(1-p)) + 1

def geom3(p):
    '''simulando ensayos Bernoulli hasta obtener un éxito'''
    i = 0
    while True:
        i += 1
        if random() < p:
            return i

In [ ]:
'''Medición de tiempos de ejecución'''     
n = 100
metodo1 = [0] * 100
metodo2 = [0] * 100
metodo3 = [0] * 100

for p in range(1, n):
    metodo1[p] = timeit("geom1(p/n)", globals = globals(), number = 10000)
    metodo2[p] = timeit("geom2(p/n)", globals = globals(), number = 10000)
    metodo3[p] = timeit("geom3(p/n)", globals = globals(), number = 10000)

In [ ]:
fig, ax1 = plt.subplots(figsize = (15,3))


t = np.arange(1, n, 1)
ax1.plot(metodo1, 'r', label = 'Transformada Inversa')
ax1.plot(metodo2, 'b', label = 'int(log(1-U)/log(1-p)) + 1')
ax1.plot(metodo3, 'g', label = 'Simulando Bernoullis')
ax1.set_xticklabels([str(i/5.) for i in range(6)])
#ax1.set_xticks([i for i in range(0,101,20)])
ax1.set_xlabel('p')
ax1.set_ylabel('tiempo por ejecución')

ax1.set_title('Métodos de simulación de geométricas')

ax1.legend(loc = 'best')

<font color = green size = 6pt>
Distribución de Poisson
</font>

In [ ]:
def Poisson(lamda):
    '''Método de transformada inversa'''
    U = random() 
    i = 0; p = exp(-lamda)
    F=p
    while U >= F:
        i += 1        
        p *=lamda / i
        F = F + p
    return i

def Poisson_ordenado(L):
    I = int(L)
    p = exp(-L)
    F = p
    ## Cálculo de F(I)
    for i in range(1,I+1):
        p *= L / i
        F += p
    u=random()
    if u>=F: #recorre I, I+1, I+2, ...
        while u>F:
            I += 1
            p *= L/I
            F += p
        return I
    else:
        while u < F: #recorre I-1, I-2, ...
            F -= p
            p *= I / L
            I -= 1
        return I + 1
    
def Poisson_con_exp(lamda):
    X = 0
    Producto = 1 - random()
    cota = np.exp(-lamda)
    while Producto >= cota:
        Producto *= 1- random()
        X += 1
    return X


In [ ]:

metodo_TI = []
metodo_TIordenado = []
metodo_exp = []
'''Medición de tiempos de ejecución'''     


for lamda in range(1, 100):
    metodo_TI.append( timeit("Poisson(lamda)", globals = globals(), number = 10000))
    metodo_TIordenado.append( timeit("Poisson_ordenado(lamda)", globals = globals(), number = 10000))
    metodo_exp.append( timeit("Poisson_con_exp(lamda)", globals = globals(), number = 10000))
    
for lamda in range(100, 300, 10):
    metodo_TI.append( timeit("Poisson(lamda)", globals = globals(), number = 10000))
    metodo_TIordenado.append( timeit("Poisson_ordenado(lamda)", globals = globals(), number = 10000))
    metodo_exp.append( timeit("Poisson_con_exp(lamda)", globals = globals(), number = 10000))

    

In [ ]:
fig, [ax1, ax2] = plt.subplots(nrows = 1, ncols = 2, figsize = (15,4))

ax1.plot(metodo_TI[:100], 'b', label = 'Transformada inversa')
ax1.plot(metodo_TIordenado[:100], 'g', label = 'Probs ordenadas')
ax1.plot(metodo_exp[:100], 'r', label = 'con exponenciales')
#ax.set_xticks([i for i in range(0,101,20)])
ax1.set_xlabel('lambda')
ax1.set_ylabel('tiempo por ejecución')

ax1.set_title('Métodos de simulación de Poisson')

ax1.legend(loc = 'best')

ax2.plot(metodo_TI[100:], 'b', label = 'Transformada inversa')
ax2.plot(metodo_TIordenado[100:], 'g', label = 'Probs ordenadas')
ax2.plot(metodo_exp[100:], 'r', label = 'con exponenciales')
ax2.set_xticklabels([str((i-1) * 25  + 100) for i in range(9)])
#ax2.set_xticks([i + 100 for i in range(0,101,20)])
ax2.set_xlabel('lambda')
ax2.set_ylabel('tiempo por ejecución')

ax2.set_title('Métodos de simulación de Poisson')

ax2.legend(loc = 'best')
